<center><div style="font-size : 20px; font-weight : bold; color : darkbue;">LAPORAN TUGAS BESAR</div>
<div>Tubes Tahap-1: Pengelompokan dg k-Means</div>
<hr>
<div>6182001003 | Rivaldi Godjali - 6182001004 | William Surya W</div>

In [52]:
#import library
import pandas as pd;
import numpy as np;

In [53]:
#load data
mall = pd.read_csv("https://raw.githubusercontent.com/shrk-sh-ioai/tubes-ppdp/main/mall-customer-dt/mall-customers.csv", sep=",", encoding='cp1252');


In [54]:
#checking data - NaN value
dfCheck = mall[mall.isna().any(axis=1)];
print("Data with NaN values :"); print(dfCheck);

Data with NaN values :
Empty DataFrame
Columns: [CustomerID, Gender, Age, Annual Income (k$), Spending Score (1-100)]
Index: []


In [55]:
#cheking data - Minimum value
print("Minimum values :"); print(mall.min());

Minimum values :
CustomerID                     1
Gender                    Female
Age                           18
Annual Income (k$)            15
Spending Score (1-100)         1
dtype: object


In [56]:
#checking data - Maximum value
print("Maximum values :"); print(mall.max());

Maximum values :
CustomerID                 200
Gender                    Male
Age                         70
Annual Income (k$)         137
Spending Score (1-100)      99
dtype: object


In [57]:
#preparing data
cuts = ["1","2","3","4","5","6","7","8","9","10"];
bins = pd.IntervalIndex.from_tuples([(0, 11), (11, 21), (21, 31), (31, 41), (41, 51), (51,61), (61,71), (71,81), (81,91), (91,101)], closed='left');

mall['Score'] = np.array(cuts)[
    pd.cut(mall["Spending Score (1-100)"],
    bins = bins).cat.codes
    ];

print(mall);

     CustomerID  Gender  Age  Annual Income (k$)  Spending Score (1-100) Score
0             1    Male   19                  15                      39     4
1             2    Male   21                  15                      81     9
2             3  Female   20                  16                       6     1
3             4  Female   23                  16                      77     8
4             5  Female   31                  17                      40     4
..          ...     ...  ...                 ...                     ...   ...
195         196  Female   35                 120                      79     8
196         197  Female   45                 126                      28     3
197         198    Male   32                 126                      74     8
198         199    Male   32                 137                      18     2
199         200    Male   30                 137                      83     9

[200 rows x 6 columns]
